<a href="https://colab.research.google.com/github/linsut/DatSci/blob/master/ESA%20KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Exercise 1

In [1]:
import numpy as np
from collections import Counter

In [2]:
def normalize(data):
  #This only works on a two dimensional example
  new_data = []
  max1 = max(data[:][0])
  min1 = min(data[:][0])
  max2 = max(data[:][1])
  min2 = min(data[:][1])
  for index in range (0, len(data)):
    new_data.append([(data[index][0] - min1) / (min1 -max1), (data[index][1] - min2) / (min2 -max2)])
    #new_data[index][1] = (data[index][1] - min2) / (min2 -max2)

  return new_data

In [3]:
def euclidian_distance(pointa, pointb):
  sum = 0
  for index in range(len(pointa)):
    sum += (pointa[index] - pointb[index])**2

  return np.sqrt(sum)


In [4]:
def get_neighbors(training_set, test_instance, labels, k):
  distances = []
  for index in range(len(training_set)):
    dist = euclidian_distance(test_instance, training_set[index])
    distances.append((training_set[index], dist, labels[index]))
  distances.sort(key=lambda x: x[1])
  neighbors = distances[:k]
  return neighbors

In [5]:
def vote(neighbors):
  class_counter = Counter()
  for neighbor in neighbors:
    class_counter[neighbor[2]] += 1
  return class_counter.most_common(1)[0][0]

An example with made-up data. Data does not stand for anything.

In [7]:
data = [[0.8, 400],[12, 134000],[0, 20000],[67, 32000]]
labels = [1, 0, 1, 0]
new_data = normalize(data)
print(new_data)
print("Result of vote: " + str(vote(get_neighbors(new_data[:-1], new_data[3], labels[:-1], 3))))

[[-0.0, -0.002895781711795086], [-0.028056112224448898, -1.0], [0.0020040080160320644, -0.14917753828701077], [-0.1658316633266533, -0.23873779741469386]]
Result of vote: 1


##Exercise 2

In [8]:
%matplotlib inline
import numpy as np
from sklearn import datasets

iris = datasets.load_iris()
iris_data = iris.data
iris_labels = iris.target

To determine the similarity between two instances, we need a distance function. In our example, the Euclidean distance is ideal:

In [9]:
def distance(instance1, instance2):
    # just in case, if the instances are lists or tuples:
    instance1 = np.array(instance1) 
    instance2 = np.array(instance2)
    
    return np.linalg.norm(instance1 - instance2)

The function 'get_neighbors returns a list with 'k' neighbors, which are closest to the instance 'test_instance':

In [12]:
def get_neighbors(training_set, 
                  labels, 
                  test_instance, 
                  k, 
                  distance=distance):
    """
    get_neighors calculates a list of the k nearest neighbors
    of an instance 'test_instance'.
    The list neighbors contains 3-tuples with  
    (index, dist, label)
    where 
    index    is the index from the training_set, 
    dist     is the distance between the test_instance and the 
             instance training_set[index]
    distance is a reference to a function used to calculate the 
             distances
    """
    distances = []
    for index in range(len(training_set)):
        dist = distance(test_instance, training_set[index])
        distances.append((training_set[index], dist, labels[index]))
    distances.sort(key=lambda x: x[1])
    neighbors = distances[:k]
    return neighbors

__Voting to get a Single Result__

We will write a vote function now. This functions uses the class 'Counter' from collections to count the quantity of the classes inside of an instance list. This instance list will be the neighbors of course. The function 'vote' returns the most common class:

In [13]:
def vote(neighbors):
    class_counter = Counter()
    for neighbor in neighbors:
        class_counter[neighbor[2]] += 1
    return class_counter.most_common(1)[0][0]

Now to classify our new data

In [14]:
neighbors = get_neighbors(iris_data, iris_labels, 
                          [[4.8,2.5,5.3,2.4]], 
                          3, 
                          distance=distance)
                              
print("Result of vote: ", iris.target_names[vote(neighbors)])

Result of vote:  virginica


'vote_prob' is a function like 'vote' but returns the class name and the probability for this class:

In [15]:
def vote_prob(neighbors):
    class_counter = Counter()
    for neighbor in neighbors:
        class_counter[neighbor[2]] += 1
    labels, votes = zip(*class_counter.most_common())
    winner = class_counter.most_common(1)[0][0]
    votes4winner = class_counter.most_common(1)[0][1]
    return winner, votes4winner/sum(votes)

The same test with our new data, with a different k this time.

In [16]:
neighbors = get_neighbors(iris_data, iris_labels, 
                          [[4.8,2.5,5.3,2.4]], 
                          5, 
                          distance=distance)

result, propability = vote_prob(neighbors)
print("Result of vote: ", iris.target_names[result], " with propability " + str(propability))

Result of vote:  virginica  with propability 1.0
